# Train functionality

In [ ]:
#| default_exp train

In [ ]:
#| export
from __future__ import annotations
import math, random, torch, matplotlib.pyplot as plt, numpy as np, matplotlib as mpl, shutil, os, gzip, pickle, re, copy, time
from pathlib import Path
from functools import partial
import fastcore.all as fc
from glob import glob
import json

from torch import tensor, nn, optim
import torch.nn.functional as F
from datasets import load_dataset
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, default_collate
from torch.nn import init
from torch.nn.utils.rnn import pad_sequence
from typing import List, Optional

from datetime import datetime, timedelta
import calendar
from fastprogress import progress_bar
from einops import rearrange

from toolken.model import *
from toolken.tokenizer import *
from toolken.datasets import *

Below is a basic training object that trains Toolken embeddings, stores the training results in a JSON file and saves the embeddings to a pytorch file.

In [ ]:
#| export
class Trainer:
    def __init__(self, dls, model, tokenizer, save_path, epochs=10, lr=1e-3, device='cuda', opt_func=optim.Adam):
        fc.store_attr()
        if not self.save_path.endswith('/'): self.save_path += '/'
    def train_embeddings(self):
        self.all_results = {}
        opt = self.opt_func([p for p in self.model.parameters() if p.requires_grad], self.lr)
        for self.epoch in progress_bar(range(self.epochs)):
            
            self.make_storage()

            # do train
            for batch in progress_bar(self.dls.train, leave=False):
                self.model.train()
                inp, label = batch[0].to(self.device), batch[1].to(self.device)
                concat_logits = self.model(inp, 0)
                loss = F.cross_entropy(concat_logits.view(-1, concat_logits.shape[-1]), label.view(-1), ignore_index=-100)
                loss.backward()
                opt.step()
                opt.zero_grad()
                self.all_results[self.epoch]['train_losses'].append(loss.detach().cpu().numpy().item())

            # do valid
            for batch in progress_bar(self.dls.valid, leave=False):
                self.model.eval()
                inp, label = batch[0].to(self.device), batch[1].to(self.device)
                concat_logits = self.model(inp, 0)
                loss = F.cross_entropy(concat_logits.view(-1, concat_logits.shape[-1]), label.view(-1), ignore_index=-100)
                self.all_results[self.epoch]['eval_losses'].append(loss.detach().cpu().numpy().item())
                self.calculate_results(concat_logits, label)
            
            self.print_epoch_results()
            self.save_embeddings()
            self.save_results()
    
    def make_storage(self):
        self.all_results[self.epoch] = {
            "train_losses": [],
            "eval_losses": [],
            "eval_matches": [],
            "eval_predicted": [],
            "eval_real": [],
            "eval_pc": []
        }
    def calculate_results(self, concat_logits, label):
        func_dict = self.model.func_dict
        pred = torch.argmax(concat_logits, dim=-1).squeeze()
        label_funcs = torch.stack([label == func_dict[op] + 32000 for op in func_dict.keys()])
        pred_funcs = torch.stack([pred == func_dict[op] + 32000 for op in func_dict.keys()])
        
        matches = torch.sum(label_funcs * pred_funcs, dim=-1).detach().cpu().numpy()
        pred_funcs = torch.sum(pred_funcs, dim=-1).detach().cpu().numpy()
        real = torch.sum(label_funcs, dim=-1).detach().cpu().numpy()
        pc = matches / (real + 1e-8)

        for key, res in zip(list(self.all_results[self.epoch].keys())[-4:], [matches, pred_funcs, real, pc]):
            self.all_results[self.epoch][key].append(res.tolist())
    def print_epoch_results(self):
        train_loss = np.array(self.all_results[self.epoch]['train_losses']).mean()
        eval_loss = np.array(self.all_results[self.epoch]['eval_losses']).mean()
        eval_pc = np.array(self.all_results[self.epoch]['eval_pc']).mean()
        print(f'Epoch {self.epoch} || Train loss: {train_loss} || Eval loss: {eval_loss} || Eval pc: {eval_pc}')
    def save_embeddings(self):
        os.makedirs(self.save_path, exist_ok=True)
        torch.save(self.model.func_embed.state_dict(), f"{self.save_path}/epoch_{self.epoch}.pth")
    def save_results(self):
        import pdb; pdb.set_trace()
        with open(f'{self.save_path}results.json', 'w') as file:
            json.dump(self.all_results, file)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

### Test

In [ ]:
# os.environ['LOCAL_RANK'] = '0'
# os.environ['WORLD_SIZE'] = '1'
# os.environ['RANK'] = '0'
# os.environ['MASTER_ADDR'] = '172.17.0.3'
# os.environ['MASTER_PORT'] = '6006'

In [ ]:
# local_rank, world_size = setup_model_parallel()

In [ ]:
# path = '../model/tokenizer.model'
# tokenizer = Tokenizer(path)

In [ ]:
# path = '/home/models/foundation/LLaMA/7B'
# checkpoint = torch.load(f'{path}/consolidated.00.pth')
# with open(Path(path) / "params.json", "r") as f: params = json.loads(f.read())
# model_args = ModelArgs(max_seq_len=2048, max_batch_size=8, **params)
# model_args.vocab_size = tokenizer.n_words
# model = Transformer(model_args).cuda().half()
# torch.set_default_tensor_type(torch.FloatTensor)
# model.load_state_dict(checkpoint, strict=False)

In [ ]:
# with open('../data/gsm8k-xl/func_dict.json', 'r') as f: func_dict = json.load(f)

In [ ]:
# fmodel = FunctionModel(model, func_dict)

In [ ]:
# len([p for p in fmodel.parameters() if p.requires_grad]) / len([p for p in fmodel.parameters()])

In [ ]:
# with open('../data/gsm8k-xl/train.json', 'r') as f: data = json.load(f)
# tds = PromptDS(data[:int(0.9*len(data))], tokenizer, func_dict)
# vds = PromptDS(data[int(0.9*len(data)):], tokenizer, func_dict)
# dls = DataLoaders(tds, vds, 1)

In [ ]:
# path = '/home/projects/toolken/'
# trainer = Trainer(dls, fmodel, tokenizer, path)

In [ ]:
# trainer.train_embeddings()